In [ ]:
## Importing Dependencies
from selenium import webdriver
from selenium_stealth import stealth
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import os
import sys
import numpy as np
from datetime import datetime, timedelta

## Initializing

In [ ]:
## Building WebScraper class
class WebScraper:
    def __init__(self, headless=True, interval=30, baseurl="https://www.capitoltrades.com/trades"):
        # Set up Chrome options
        chrome_options = webdriver.ChromeOptions()
        
        # Disables the AutomationControlled feature, which helps to prevent websites from detecting that the browser is being controlled by automation software.
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        
        # Runs Chrome in headless mode, which means it operates without a graphical user interface. This is useful for running automated tests or web scraping tasks on servers without a display.
        if headless:
            chrome_options.add_argument("--headless")
        
        # Disables the sandbox security feature. This can help avoid certain security restrictions but should be used with caution as it reduces the security of the browser.
        chrome_options.add_argument("--no-sandbox")
        
        # Disables the use of /dev/shm (shared memory) in Chrome. This can help prevent issues related to limited shared memory space, especially in containerized environments like Docker.
        chrome_options.add_argument("--disable-dev-shm-usage")
        
        # Disables GPU hardware acceleration. This can help avoid rendering issues and improve stability, particularly when running in headless mode.
        chrome_options.add_argument("--disable-gpu")
        
        # Initialize the Chrome WebDriver with ChromeDriverManager
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=chrome_options)
        
        # Apply stealth settings
        stealth(self.driver,
                languages=["en-US", "en"],
                vendor="Google Inc.",
                platform="Win32",
                webgl_vendor="Intel Inc.",
                renderer="Intel Iris OpenGL Engine",
                fix_hairline=True)
        
        self.wait = WebDriverWait(self.driver, 10)
        self.baseurl = baseurl
        self.seen_trades = set()
        self.interval = interval

    def navigate_to_site(self, baseurl=None):
        if baseurl is None:
            baseurl = self.baseurl
        self.driver.get(baseurl)
        time.sleep(5)
        try:
            self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "site-main")))  
            print("Successfully loaded the page")
        except TimeoutException:
            print("Failed to load the page")
            self.driver.quit()

    def get_page_number(self):
        try:
            find_page_max = self.driver.find_element(By.XPATH, '/html/body/div/main/main/section/div[3]/div[2]/div[1]/p[1]/b[2]')
            page_max = int(find_page_max.text)
            return page_max
        except TimeoutException:
            print("Failed to locate the page maximum")
            self.driver.quit()
            return None
        except ValueError:
            print("Failed to convert page number to integer")
            self.driver.quit()
            return None

    def extract_trades(self):
        try:
            self.wait.until(EC.presence_of_element_located((By.TAG_NAME, "tbody")))
            table_xpath = '/html/body/div/main/main/section/div[3]/div[1]/div/table'
            table = self.driver.find_element(By.XPATH, table_xpath)
            table_data = table.text
            return table_data
        except TimeoutException:
            print("Failed to locate the trades table")
            self.driver.quit()
            return None

    def close(self):
        self.driver.quit()


## Webscraping

In [ ]:
# Get page max
scraper = WebScraper(headless=False)
scraper.navigate_to_site()
page_max = scraper.get_page_number()


In [ ]:
sys.path.append(os.path.abspath(os.path.join('..')))
from db.utils import DatabaseConnector
db = DatabaseConnector(host="localhost", database="mydb", user="myuser", password="mypassword", port=5432)
db.connect()
last_row = db.get_last_row()
last_row = last_row[1:]
all_trades = []


In [ ]:
last_row_df = pd.DataFrame(last_row).T
last_row_df.head()

In [ ]:
value_to_check = last_row_df.iloc[0,-1]
value_to_check

In [ ]:
type(last_row_df.index[0])

In [ ]:
# Get trade data
try:
    if page_max is not None:
        for i in range(1, page_max + 1):
            scraper.navigate_to_site(baseurl=f"https://www.capitoltrades.com/trades?pageSize=100&page={i}")
            trades = scraper.extract_trades()
            print(f"Extracted trades from page {i}")
            if trades:
                all_trades.append(trades)
                print("Received trades")

            # Split data data
            all_trades_data = [trade.split('\n') for trade in all_trades]
            print("Split data")

            # Flatten list of lists into a single list
            all_trades_flat = [item for sublist in all_trades_data for item in sublist]

            # Remove headers
            cleaned_trades = []
            header_names = ['POLITICIAN', 'TRADED ISSUER', 'PUBLISHED', 'TRADED', 'FILED AFTER', 'OWNER', 'TYPE', 'SIZE', 'PRICE','Goto trade detail page.']
            for trade in all_trades_flat:
                if trade not in header_names:
                    cleaned_trades.append(trade)

            cleaned_trades_length = len(cleaned_trades)

            # Reshape data to 14 columns to fit formatting schema
            cleaned_trades_table = np.array(cleaned_trades).reshape(-1, 14)

            # Create header names and dataframe
            headers = ['Politician', 'Party', 'Traded Company Name', 'Traded Company Ticker', 'Published Date', 'Published Year', 'Traded On Date', 'Traded On Year', 'UoM of Time After Trade Filed', 'Time After Trade Filed', 'Trade Owner', 'Trade Type', 'Trade Size', 'Stock Price']
            cleaned_trades_df = pd.DataFrame(cleaned_trades_table, columns = headers)

            cleaned_trades_df.tail()

            ## Data transformation
            # Combining Published Date and Published Year into a single column then removing the original columns
            cleaned_trades_df["Published Datetime"] = cleaned_trades_df["Published Date"] + " " + cleaned_trades_df["Published Year"]
            cleaned_trades_df.drop(columns=["Published Date", "Published Year","UoM of Time After Trade Filed"], inplace=True)

            # Find values where word "Yesterday" is present - replace yesterday string with Yesterday's date
            yesterday_date = ( datetime.now()- timedelta(1)).strftime('%d, %b, %Y')
            cleaned_trades_df["Published Datetime"].replace("Yesterday", yesterday_date, inplace=True, regex=True)

            # Find values where word "Today" is present - replace today string with Yesterday's date
            today_date = (datetime.now()).strftime('%d, %b, %Y')
            cleaned_trades_df["Published Datetime"].replace("Today", today_date, inplace=True, regex=True)

            # Convert "Published Datetime" to mm-dd-yyyy format, no times
            def date_convert(date_to_convert):
                return datetime.strptime(date_to_convert, '%d, %b, %Y').strftime('%Y-%m-%d')

            def datetime_convert(date_convert):
                return datetime.strptime(date_convert,'%H:%M %d, %b, %Y').strftime('%Y-%m-%d')
            
            
            # Iterate through each row in "Published Datetime" to remove time from Published Datetime data (not relevant)
            for index, row in cleaned_trades_df.iterrows():
                date_str = row["Published Datetime"]
                try:
                    # Try to run date_convert - use at instead of loc if it breaks
                    cleaned_trades_df.loc[index, "Published Datetime"] = date_convert(date_str)
                except ValueError:
                    try:
                        # If date_convert fails, try to run datetime_convert - use at instead of loc if it breaks
                        cleaned_trades_df.loc[index, "Published Datetime"] = datetime_convert(date_str)
                    except ValueError:
                        # If both fail, move to the next row
                        continue

            
            # Combining Traded Datetime data and removing original columns
            cleaned_trades_df["Traded Datetime"] = cleaned_trades_df["Traded On Date"] + " " + cleaned_trades_df["Traded On Year"]
            cleaned_trades_df.drop(columns = ['Traded On Date', 'Traded On Year'], inplace=True)

            # Convert Time After Trade Filed to Int
            cleaned_trades_df["Time After Trade Filed"] = cleaned_trades_df["Time After Trade Filed"].astype(int)

            # Convert Time After Trade Filed to date, rename column
            for index, row in cleaned_trades_df.iterrows():
                days_ago = row["Time After Trade Filed"]
                cleaned_trades_df.loc[index, "Time After Trade Filed"] = (datetime.now() - timedelta(days=days_ago)).strftime('%d %b %Y')
            
            cleaned_trades_df.rename(columns = {"Time After Trade Filed": "Trade Filed Date"}, inplace=True)

            # Create Trade ID for checking duplicates
            cleaned_trades_df["Trade ID"] = (
            cleaned_trades_df["Politician"].astype(str) + "_" +
            cleaned_trades_df["Party"].astype(str) + "_" +
            cleaned_trades_df["Traded Company Name"].astype(str) + "_" +
            cleaned_trades_df["Traded Company Ticker"].astype(str) + "_" +
            cleaned_trades_df["Trade Filed Date"].astype(str) + "_" +
            cleaned_trades_df["Trade Owner"].astype(str) + "_" +
            cleaned_trades_df["Trade Type"].astype(str) + "_" +
            cleaned_trades_df["Trade Size"].astype(str) + "_" +
            cleaned_trades_df["Stock Price"].astype(str) + "_" +
            cleaned_trades_df["Published Datetime"].astype(str) + "_" +
            cleaned_trades_df["Traded Datetime"].astype(str)
        )
            
            ## If you can match an index with last row, return true and pull everything on that row and below
            # Check if the value in last_row_df[11] is in cleaned_trades_df["Trade ID"]
            value_to_check = last_row_df.iloc[0,-1]
            is_value_present = value_to_check in cleaned_trades_df["Trade ID"].values
            print(f"Is the value '{value_to_check}' present in 'Trade ID' column? {is_value_present}")

            if is_value_present:
                # Find the last occurrence of the value
                row_above_first_occurence_index = (cleaned_trades_df[cleaned_trades_df["Trade ID"] == value_to_check].index[0]) - 1

                # Cut the DataFrame to include all values above the last found match
                cleaned_trades_df = cleaned_trades_df.loc[:max(row_above_first_occurence_index,0)]

                print("DataFrame after cutting:")
                print(cleaned_trades_df)


            if i == page_max + 1:
                print("Page max reached")
                exit
                

except Exception as e:
    print(e)
    for trade in all_trades:
        print(trade)
    exit
exit